# Sample Predictions By Genre and User_ID

This is a notebook that makes predictions and stores the results so the web app can fetch the pre-processed recommendations. This also contains lists of top-rated books in some common genres.

In [1]:
import numpy as np
import pandas as pd
from surprise_functions import get_n_rec_user
from surprise import dump
import pickle

In [2]:
_,model = dump.load('model_dump_file')

test = pd.read_csv('goodreads2/testset.csv',index_col=0)
user_df = pd.read_csv('goodreads2/user_w_uid_bavg_titles.csv')
div_m = pd.read_csv('goodreads2/div_array.csv')

C:\Users\shinj\Anaconda3\envs\insight\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
user_df.author.fillna('',inplace=True)
user_df.title.fillna('',inplace=True)

In [4]:
rec = get_n_rec_user(user_df,222,model,div_m,test)

In [5]:
rec

['Malgudi Days by R.K. Narayan, Jhumpa Lahiri',
 'Crooked Kingdom by Leigh Bardugo',
 "India After Gandhi: The History of the World's Largest Democracy by Ramachandra Guha",
 'Blue Lily, Lily Blue by Maggie Stiefvater',
 'Citizen: An American Lyric by Claudia Rankine']

# Lists of books by genre

First let's do the easy part and create lists of top-rated book by genre.

In [6]:
genres = ['science-fiction',
 'fantasy',
 'young-adult',
 'historical-fiction',
 'graphic-novels','adult','fiction','mystery']

In [7]:
genre_dict = {}

for gn in genres:
    gen_book = user_df.loc[(user_df.genre==gn)&(user_df.is_div==1)&(user_df.title!='')].groupby('bid')[['title','author','b_average_rating']].first().reset_index().sort_values('b_average_rating',ascending=False).head(20).reset_index()
    genre_dict[gn] = gen_book[['title','author','b_average_rating']].rename(columns = {'title':'Title','author':'Author','b_average_rating':'Average Rating'})

In [8]:
genre_dict['adult'] = genre_dict['fiction']

In [9]:
genre_dict['any'] = pd.concat([genre_dict['adult'],genre_dict['fantasy'],genre_dict['young-adult'],genre_dict['historical-fiction'],genre_dict['mystery'],genre_dict['graphic-novels']]).sort_values('Average Rating',ascending=False)

In [10]:
genre_dict['science-fiction']['Title'] = genre_dict['science-fiction'].Title.str.replace('オール・ユー・ニード・イズ・キル','All You Need Is Kill')

In [11]:
#pickle.dump(genre_dict, open( "genrelist.p", "wb" ) )

# Prediction for a sample of usernames

Let's make and store predictions for a set of sample users.

In [12]:
names = ["Brianne",'Shinjini','Allen','Emily','Adam','Donald','Kim',"Ilia","Bennett","Yaarit","Brandon","Steven","John","Randolph","Alex","Gary","Julia","Ilia","Chang","Daniel","Kai","Jason","Victoria","Antoine","Wenke","Avi","Jesse","Anna","Nina","Paul","Avery"]

In [13]:
names = pd.Series(names).drop_duplicates().tolist()

In [14]:
len(names)

30

In [16]:
import random

recs = []

uid = list(range(1517))
ul = random.sample(uid,30)

for uid in ul:
    recs.append(get_n_rec_user(user_df,uid,model,div_m,test))

In [19]:
name_rec = dict(zip(names,recs))

In [20]:
#pickle.dump(name_rec, open( "name_rec.p", "wb" ) )